1. Setup + Imports

In [3]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np

2. Data Preparation
    - loading dataset
    - preprocessing
    - creating sequences
    - splitting data

In [11]:
df = pd.read_csv("data/train_delay_data.csv")
print(df.shape)
df.head()


(2878, 7)


,Distance Between Stations (km),Weather Conditions,Day of the Week,Time of Day,Train Type,Historical Delay (min),Route Congestion
0,100,Clear,Monday,Morning,Express,5,Low
1,150,Rainy,Tuesday,Afternoon,Superfast,10,Medium
2,200,Foggy,Wednesday,Evening,Local,15,High
3,50,Clear,Thursday,Night,Express,2,Low
4,75,Rainy,Friday,Morning,Superfast,8,Medium


3. Model set up
    - Define LSTM architecture
    - Initialize model, optimizer, and loss function


4. Training Loop
    - Train model over epochs
    - Track loss and accuracy
    - Save best model (optional)

5. Evaluation & Results
    - Evaluate on test set
    - Plot training/validation loss curves
    - Visualize predicted vs actual delays
